In [18]:
import math

from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import scipy.stats as st
import scipy.optimize as optimize
import enum

In [17]:
import OptionLib as opt

df = opt.yf_get_chains('JPM')[:10]
df


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expDate,daysToExp
0,JPM230421C00065000,2023-04-04 19:47:43+00:00,65.0,63.10,0.0,0.0,0.0,0.0,4.0,0.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
1,JPM230421C00070000,2023-04-11 19:07:19+00:00,70.0,58.70,0.0,0.0,0.0,0.0,61.0,31.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
2,JPM230421C00075000,2023-04-04 19:48:33+00:00,75.0,53.15,0.0,0.0,0.0,0.0,5.0,0.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
3,JPM230421C00080000,2023-04-17 13:49:40+00:00,80.0,58.65,0.0,0.0,0.0,0.0,2.0,5.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
4,JPM230421C00085000,2023-04-04 18:23:49+00:00,85.0,43.10,0.0,0.0,0.0,0.0,83.0,0.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
5,JPM230421C00090000,2023-04-04 18:23:31+00:00,90.0,37.90,0.0,0.0,0.0,0.0,89.0,0.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
6,JPM230421C00095000,2023-04-14 15:08:49+00:00,95.0,43.80,0.0,0.0,0.0,0.0,1.0,1.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
7,JPM230421C00100000,2023-04-18 16:04:30+00:00,100.0,41.02,0.0,0.0,0.0,0.0,5.0,33.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
8,JPM230421C00105000,2023-04-18 19:47:53+00:00,105.0,36.75,0.0,0.0,0.0,0.0,2.0,140.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3
9,JPM230421C00110000,2023-04-14 18:31:12+00:00,110.0,28.46,0.0,0.0,0.0,0.0,8.0,42.0,0.00001,True,REGULAR,USD,call,2023-04-21 23:59:59,3


In [28]:
#opt.delta(r=0.07, spot=a, strike=b, time=c, sigma=d, type=e)
r = 0.05
spot =  139.92
df1 = df.apply(lambda x: opt.delta(r=r, spot=spot, strike=x[2], time=x[16], sigma=x[10], type=x[14]), axis=1)

In [29]:
df1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
dtype: float64

In [ ]:
x = df.iloc[0]
opt.delta(r=0.07, spot=spot, strike=x[2], time=x[16], sigma=x[10], type=x[14])

In [ ]:
def d1(r, spot, strike, time, sigma):
    return (np.log(spot / strike) + (r + sigma ** 2 / 2) * time) / (sigma * np.sqrt(time))

def d2(r, spot, strike, time, sigma):
    return d1(r, spot, strike, time, sigma) - sigma * np.sqrt(time)

In [ ]:
def delta(r, spot, strike, time, sigma, type):
    """
    Returns Delta greek of an option given
    """
    d1_value = d1(r, spot, strike, time, sigma)

    try:
        if type == 'call':
            delta_value = norm.cdf(d1_value, 0, 1)
        elif type == 'put':
            delta_value = -norm.cdf(-d1_value, 0, 1)

        return delta_value

    except:
        raise 'oops'

# Heston Delta
Addressing the https://quant.stackexchange.com/questions/61849/delta-of-an-option-under-heston-model
For Heston we got the following equation: $\frac{dC}{dS_0} = e^{-q(T-t)}P_1 > 0$ \
$\Delta_{C, t} = \frac{C_t}{S_t} = e^{-q(T-t)}P_1, $ and as follows from Put-Call parity $\Delta_{P, t} = \frac{P_t}{S_t} = \frac{C_t}{S_t} - e^{-q(T-t)} = e^{-q(T-t)}(P_1 - 1) $

In [ ]:
def heston_delta(self, q=0, a=0.00001, b=100, n=10000):
        P1 = self.probability(a, b, n)[0]

        return {
            'call': math.e**(-q * self.tau) * P1,
            'put':  math.e**(-q * self.tau) * (P1 - 1)
            }

In [5]:
import OptionLib as opt

In [15]:
heston_model = opt.Heston(S0=100, strike=100, time=1.0, r=0.05, kappa=0.0, theta=0.01, v0=0.01, lamda=0.0, sigma=0.000001, rho=-0.7)

h_delta = heston_model.heston_delta()
h_delta

{'call': 0.7088406167635928, 'put': -0.2911593832364072}

# SABR Delta

# 1

In [14]:
import math
import scipy.stats as si
import sympy as sy

def bachelier(market_vol, spot, strike, forward, time, option):
    if option == 'call':
        d = (spot * np.exp(forward * time) - strike) / np.sqrt(
            market_vol ** 2 / (2 * forward) * (np.exp(2 * forward * time) - 1))

        p = np.exp(-forward * time) * (spot * np.exp(forward * time) - strike) * si.norm.cdf(d) + \
            np.exp(-forward * time) * np.sqrt(
            market_vol ** 2 / (2 * forward) * (np.exp(2 * forward * time) - 1)) * si.norm.pdf(d)
    if option == 'put':
        p = bachelier(market_vol, spot, strike, forward, time, 'call') - spot + np.exp(-forward * time) * strike

    return p

def deltaSabrBachelier(forward, strike, spot, time, market_vol, option):
    return sy.diff(bachelier(forward, strike, spot, time, market_vol, option), spot)

# 2

In [ ]:
"""
# Barret's Delat page 4'
skew = beta*alpha*F**(1-beta) # + O(epsilon)
delta_sabr = delta_bsm + vega_bsm*skew
"""

def delta_sabr(alpha, beta, F, r, spot, strike, time, sigma, type):
    skew = beta*alpha*F**(1-beta) # + O(epsilon)
    delta_sabr_value = opt.delta(r, spot, strike, time, sigma, type) + opt.vega(r, spot, strike, time, sigma)*skew # delta_bsm + vega_bsm*skew
    return  delta_sabr_value

# Delta Hedge

The delta, à, of an option or a portfolio of options is the sensitivity of the option or portfolio to the underlying. It is the rate of change of value with respect to the asset:

$\Delta = \frac{dV}{dS}$

Here V can be the value of a single contract or of a whole portfolio of contracts. The delta of a portfolio of options is just the sum of the deltas of all the individual positions.
The theoretical device of delta hedging, introduced in Chapter 5, for eliminating risk is far more than that, it is a very important practical technique.
Roughly speaking, the financial world is divided up into speculators and hedgers. The spec- ulators take a view on the direction of some quantity such as the asset price (or more abstract quantities such as volatility) and implement a strategy to take advantage of their view. Such people may not hedge at all.
Then there are the hedgers. There are two kinds of hedger: the ones who hold a position already and want to eliminate some very specific risk (usually using options) and the ones

elling (or buying) the options because they believe they have a better price and can make money by hedging away all risk. It is the latter type of hedger that is delta hedging. They can only guarantee to make a profit by selling a contract for a high value if they can eliminate all of the risk due to the random fluctuation in the underlying.
Delta hedging means holding one of the option and short a quantity à of the underlying. Delta can be expressed as a function of S and t, I’ll give some formulae later in this section. This function varies as S and t vary. This means that the number of assets held must be continuously changed to maintain a delta-neutral position; this procedure is called dynamic hedging. Changing the number of assets held requires the continual purchase and/or sale of the stock. This is called rehedging or rebalancing the portfolio.
This delta hedging may take place very frequently in highly liquid markets where it is relatively costless to buy and sell. Thus the Black–Scholes assumption of continuous hedging may be quite accurate. In less liquid markets, you lose a lot on bid-offer spread and will therefore hedge less frequently. Moreover, you may not even be able to buy or sell in the quantities you want. Even in the absence of costs, you cannot be sure that your model for the underlying is accurate. There will certainly be some risk associated with the model. These issues make delta hedging less than perfect and in practice the risk in the underlying cannot be hedged away perfectly. Issues of discrete hedging and transaction costs are covered in depth in Chapters 47 and 48.
Some contracts (see especially Chapter 23) have a delta that becomes very large at special times or asset values. The size of the delta makes delta hedging impossible; what can you do if you find yourself with a theoretical delta requiring you to buy more stock than exists? In such a situation the basic foundation of the Black–Scholes world has collapsed and you would be right to question the validity of any pricing formula. This happens at expiry close to the strike for binary options. Although I’ve given a formula for their price above and a formula for their delta below, I’d be careful using them if I were you.


Continuous time hedging is for the
textbook.
The delta does not
work on a portfolio of options that mixes longs and shorts.
It is an extremely weak measure of risks.

Use a discrete delta with increments. Shadow delta adds
some vegas and gammas to it.


If Position Delta is Negative, Hedge With These Strategies:
> Long Stock
 Long Calls
 Short Puts
 Long Call Spreads
 Short Put Spreads


If Position Delta is Positive, Hedge With These Strategies:
> Short Stock
Long Puts
 Short Calls
 Short Call Spreads
 Long Put Spreads